In [13]:
import os
import json

import pandas as pd

import locale
locale.setlocale(locale.LC_TIME, "nl_NL.UTF-8")

'nl_NL.UTF-8'

In [69]:
base_path = "../data"

In [118]:
df = pd.DataFrame()
for location in os.listdir(base_path):
    
    loc_path = os.path.join(base_path, location)
    for year in os.listdir(loc_path):
        
        year_path = os.path.join(loc_path, year)
        for file in os.listdir(year_path):
            
            file_path = os.path.join(year_path, file)

            # Read file
            file_df = pd.read_json(file_path)
            file_df = file_df.rename(columns={"id": "heat_id", "name": "heat_name"})
        
            # Explode drivers list
            file_df = file_df.explode("drivers", ignore_index=True)
            file_df = file_df.drop(columns=["drivers"]).join(pd.json_normalize(file_df["drivers"]))
            
            # Explode lap times
            file_df = file_df.explode("lap_times", ignore_index=True)
            file_df["lap"] = file_df.groupby(["heat_id", "kart"]).cumcount() + 1
            
            file_df = file_df.dropna()
            file_df["year"] = year
            file_df["location"] = location
            
            df = pd.concat([df, file_df], ignore_index=True)
            
# change datatypes
df["datetime"] = df["year"] + ' ' + df["datetime"]
df["datetime"] = pd.to_datetime(df["datetime"], format="%Y %A %d %B %H:%M")
df = df.astype({"kart": "int", "lap_times": "float"})
df = df.drop(["year"], axis=1)
df = df.rename(columns={"lap_times": "lap_time"})

In [119]:
df.head()

,heat_id,heat_name,datetime,name,kart,lap_time,lap,location
0,523341BC2F7E42F4A3976495919C6D4D,Heat 1,2025-02-01 12:00:00,Tymon Keler,75,54.89,1,delft
1,523341BC2F7E42F4A3976495919C6D4D,Heat 1,2025-02-01 12:00:00,Tymon Keler,75,40.19,2,delft
2,523341BC2F7E42F4A3976495919C6D4D,Heat 1,2025-02-01 12:00:00,Tymon Keler,75,39.14,3,delft
3,523341BC2F7E42F4A3976495919C6D4D,Heat 1,2025-02-01 12:00:00,Tymon Keler,75,39.05,4,delft
4,523341BC2F7E42F4A3976495919C6D4D,Heat 1,2025-02-01 12:00:00,Tymon Keler,75,37.03,5,delft


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094581 entries, 0 to 1094580
Data columns (total 8 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   heat_id    1094581 non-null  object        
 1   heat_name  1094581 non-null  object        
 2   datetime   1094581 non-null  datetime64[ns]
 3   name       1094581 non-null  object        
 4   kart       1094581 non-null  int64         
 5   lap_time   1094581 non-null  float64       
 6   lap        1094581 non-null  int64         
 7   location   1094581 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 66.8+ MB


In [121]:
df.to_csv("data.csv", index=False)

In [111]:
test = pd.read_json("../data/delft/2025/Januari.json")
test = test.rename(columns={"id": "heat_id", "name": "heat_name"})

In [112]:
# 1 row = 1 heat
test.head()

,heat_id,heat_name,datetime,drivers
0,CD1BB82C5C0840E5882915EF0DC02E5A,Heat 13,woensdag 1 januari 14:00,"[{'name': 'Mohammed Amine', 'kart': '39', 'lap..."
1,E91DD4299133459C926E6C4DFDB74B24,Heat 15,woensdag 1 januari 14:30,"[{'name': 'Michael Reibert', 'kart': '40', 'la..."
2,90F1B483147E4BD5A30F68DCECCB3214,Heat 16,woensdag 1 januari 14:45,"[{'name': 'Aryan Ford', 'kart': '78', 'lap_tim..."
3,995D7DEF76E2405698BC5319ABC9D72F,Heat 17,woensdag 1 januari 15:00,"[{'name': 'Matei', 'kart': '34', 'lap_times': ..."
4,1FE278FF47194583982BCCF185154927,Heat 18,woensdag 1 januari 15:15,"[{'name': 'Michael Reibert', 'kart': '2', 'lap..."


In [113]:
test = test.explode("drivers", ignore_index=True)
test = test.drop(columns=["drivers"]).join(pd.json_normalize(test["drivers"]))

In [114]:
# 1 row = 1 heat + 1 driver
test.head()

,heat_id,heat_name,datetime,name,kart,lap_times
0,CD1BB82C5C0840E5882915EF0DC02E5A,Heat 13,woensdag 1 januari 14:00,Mohammed Amine,39,"[37.75, 34.31, 34.57, 36.84, 35.44, 33.88, 74...."
1,CD1BB82C5C0840E5882915EF0DC02E5A,Heat 13,woensdag 1 januari 14:00,Bilal Hachti,17,"[59.43, 50.9, 49.93, 44.89, 65.87, 43.15, 41.5..."
2,CD1BB82C5C0840E5882915EF0DC02E5A,Heat 13,woensdag 1 januari 14:00,Mouaad Hachti,2,"[150.92, 81.98, 43.38, 41.55, 72.03, 39.73, 41..."
3,E91DD4299133459C926E6C4DFDB74B24,Heat 15,woensdag 1 januari 14:30,Michael Reibert,40,"[38.76, 33.95, 30.41, 30.49, 28.89, 31.0, 28.9..."
4,E91DD4299133459C926E6C4DFDB74B24,Heat 15,woensdag 1 januari 14:30,Amber Koot,1,"[31.45, 32.79, 29.34, 28.84, 29.5, 29.85, 28.8..."


In [115]:
test = test.explode("lap_times", ignore_index=True)
test["lap"] = test.groupby(["heat_id", "kart"]).cumcount() + 1

In [117]:
# 1 row = 1 heat + 1 driver + 1 lap
test.head()

,heat_id,heat_name,datetime,name,kart,lap_times,lap
0,CD1BB82C5C0840E5882915EF0DC02E5A,Heat 13,woensdag 1 januari 14:00,Mohammed Amine,39,37.75,1
1,CD1BB82C5C0840E5882915EF0DC02E5A,Heat 13,woensdag 1 januari 14:00,Mohammed Amine,39,34.31,2
2,CD1BB82C5C0840E5882915EF0DC02E5A,Heat 13,woensdag 1 januari 14:00,Mohammed Amine,39,34.57,3
3,CD1BB82C5C0840E5882915EF0DC02E5A,Heat 13,woensdag 1 januari 14:00,Mohammed Amine,39,36.84,4
4,CD1BB82C5C0840E5882915EF0DC02E5A,Heat 13,woensdag 1 januari 14:00,Mohammed Amine,39,35.44,5
